# Lab.4.05  lab-comparing-regression-models

In [211]:
# 1.Import the necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from sklearn.preprocessing import StandardScaler
import datetime

In [212]:
from sklearn.model_selection import train_test_split

In [213]:
# 2.Load the we_fn_use_c_marketing_customer_value_analysis.csv into the variable customer_df.
customer_df = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [214]:
# 3.Rename the columns so they follow the PE8 (snake case)
cols = [i.lower() for i in customer_df.columns]
customer_df.columns = cols 
cols = [i.replace(' ','_') for i in customer_df.columns]
customer_df.columns = cols 


In [215]:
target = customer_df['total_claim_amount']
customer_df = customer_df.drop(['customer','total_claim_amount'],axis=1)

In [216]:
# 4.Load the continuous and discrete variables into continuous_df and discrete_df variables.
customer_df_cat = customer_df.select_dtypes(object)
customer_df_num = customer_df.select_dtypes(np.number)
dis = []
cont = [] 

for i in customer_df_num.columns:
    if customer_df_num[i].nunique() > 100:
        cont.append(i)
    else:
        dis.append(i)

In [217]:
#Copy of the data frame
customer_df_w = customer_df.copy()

In [219]:
#Dividing the variables.
customer_df_cat = customer_df_w.select_dtypes(object).copy()
customer_df_num = customer_df_w.select_dtypes(np.number)

In [223]:
# 3.Normalize the  numerical variables.
transformer = StandardScaler().fit(customer_df_num)
customer_df_normalized = transformer.transform(customer_df_num)
customer_df_num_t = pd.DataFrame(customer_df_normalized, columns = customer_df_num.columns)

In [224]:
# 4.Encode the categorical variables

In [225]:
customer_df_cat['coverage'] = customer_df_cat['coverage'].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
customer_df_cat['employmentstatus'] = customer_df_cat['employmentstatus'].map({"Employed" : 4, "Unemployed" : 0, "Medical Leave" : 2,"Disabled" : 1,"Retired" : 3})
customer_df_cat['location_code'] = customer_df_cat['location_code'].map({"Suburban" : 0, "Rural" : 1, "Urban" : 2})
customer_df_cat['vehicle_size'] = customer_df_cat['vehicle_size'].map({"Small" : 0, "Medsize" : 1, "Large" : 2})
customer_df_cat_lab = customer_df_cat[['coverage','employmentstatus','location_code','vehicle_size']]

In [226]:
customer_df_cat_onehot = customer_df_cat.drop(['coverage','employmentstatus','location_code','vehicle_size'],axis=1)

customer_df_cat_hot = pd.get_dummies(customer_df_cat_onehot)
customer_df_cat_hot.head()

,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,response_No,response_Yes,education_Bachelor,education_College,education_Doctor,...,sales_channel_Agent,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Four-Door Car,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
0,0,0,0,0,1,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,1,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [227]:
# 5.The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
customer_df_cat['effective_to_date'] = pd.to_datetime(customer_df_cat['effective_to_date'])
customer_df_cat['effective_to_date'] = customer_df_cat['effective_to_date'].dt.strftime('%d%m')
customer_df_cat['effective_to_date'] = pd.to_numeric(customer_df_cat['effective_to_date'])

In [228]:
# 6.Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.
categorical = pd.concat([customer_df_cat_lab,customer_df_cat_hot,customer_df_cat['effective_to_date']], axis=1)
categorical.shape

(9134, 107)

In [232]:
customer = pd.concat([categorical, customer_df_num_t],axis=1)
customer.head()

,coverage,employmentstatus,location_code,vehicle_size,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,response_No,...,vehicle_class_Sports Car,vehicle_class_Two-Door Car,effective_to_date,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0,4,0,1,0,0,0,0,1,1,...,0,1,2402,-0.762878,0.612827,-0.703925,1.678099,-1.543287,-0.42225,-0.822648
1,1,0,0,1,1,0,0,0,0,1,...,0,0,3101,-0.149245,-1.239617,0.022691,-0.208186,-0.217334,-0.42225,2.106160
2,2,4,0,1,0,0,1,0,0,1,...,0,1,1902,0.710636,0.365710,0.429596,0.288205,-0.360680,-0.42225,-0.404247
3,0,0,0,1,0,1,0,0,0,1,...,0,0,2001,-0.052263,-1.239617,0.371467,0.288205,0.606907,-0.42225,1.687759
4,0,4,1,1,0,0,0,0,1,1,...,0,0,302,-0.755575,0.203390,-0.587666,-0.307465,-0.145661,-0.42225,-0.822648


## 1.In this final lab, we will model our data. Import sklearn train_test_split and separate the data.

In [233]:
y = target
X = customer

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

## 2.Try a simple linear regression with all the data to see whether we are getting good results.

In [235]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [236]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_test)
r2_score(y_test, predictions)

0.60265827029233

## 3.Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.


## 4.Use the function to check LinearRegressor and KNeighborsRegressor.

In [237]:
from sklearn.neighbors import KNeighborsRegressor

KNN = KNeighborsRegressor(n_neighbors=3)
KNN.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [238]:
score = KNN.score(X_test, y_test)
score

0.41723530162751665